In [1]:
import pandas as pd
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np 
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch
from tqdm import tqdm
from numpy import array
import random

In [2]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())

PyTorch version:
1.4.0
GPU Detected:
True


In [3]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [4]:
# Linear Neural Network class
class MLP(nn.Module):
    def __init__(self, input_size, num_hidden, hidden_dim, dropout):
        super(MLP, self).__init__()
        self.hidden_layers = nn.ModuleList([])
        self.hidden_layers.append(nn.Linear(input_size, hidden_dim))
        for i in range(num_hidden - 1):
            self.hidden_layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.dropout = nn.Dropout(dropout)
        self.output_projection = nn.Linear(hidden_dim, 1)
        self.nonlinearity = nn.ReLU()

    def forward(self, x):
        for hidden_layer in self.hidden_layers:
            x = hidden_layer(x)
            x = self.dropout(x)
            x = self.nonlinearity(x)
        out = self.output_projection(x)
        return out

In [5]:
# instantiate the class with params and move it to GPU if available
newmodel = MLP(856, 3, 256, 0.5).double()
newmodel.to(device)

MLP(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=856, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=256, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (output_projection): Linear(in_features=256, out_features=1, bias=True)
  (nonlinearity): ReLU()
)

In [20]:
# get files from the train split
files=pd.read_json('../../train_files_801010.json')

In [21]:
len(files)

159742

In [10]:
# split the input features and the labels in the train set
def split_sequences(sequences):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + 1
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
# split the training files into a train set and val set
random.seed(42)
np.random.seed(42)
train_set=random.choices(list(files[0]),k=int(len(files)*0.8))
val_set=list(set(files[0])-set(train_set))

In [23]:
# parameters
numberepochs=10
breakpoint=50
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(newmodel.parameters(), lr=1e-1)
train_episodes = 500
batch_size = 100
epochs = 1
counter = 0
print_every = 50
clip = 5
valid_loss_min = np.Inf

# training the model over 10 epochs
for i in range(numberepochs):
    epoch_train=random.choices(train_set,k=750*8)
    epoch_val=random.choices(val_set,k=750*2)


    for number in range(int(750*8/breakpoint)):
        trainingnp_x= np.empty((0,1,856), int)
        trainingnp_y= np.empty((0,), int)
        startno=number*50
        for i in tqdm(epoch_train[startno:startno+50]):
            joineddf=pd.read_feather('../../processed3-edited/'+i)
            joineddf=joineddf.fillna(0)
            tnp=joineddf[[c for c in joineddf if c not in ['Retweets']] 
                   + ['Retweets']].to_numpy()
            trainingnpx,trainingnpy=split_sequences(tnp)

            trainingnp_x = np.append(trainingnp_x, trainingnpx, axis=0)
            trainingnp_y = np.append(trainingnp_y, trainingnpy, axis=0)

        valnp_x= np.empty((0,1,856), int)
        valnp_y= np.empty((0,), int)
        for i in tqdm(epoch_val[startno:startno+50]):
            joineddf=pd.read_feather('../../processed3-edited/'+i)
            joineddf=joineddf.fillna(0)
            vnp=joineddf[[c for c in joineddf if c not in ['Retweets']] 
                   + ['Retweets']].to_numpy()
            valnpx,valnpy=split_sequences(tnp)

            valnp_x = np.append(valnp_x, valnpx, axis=0)
            valnp_y = np.append(valnp_y, valnpy, axis=0)
        train_data = TensorDataset(torch.from_numpy(trainingnp_x), torch.from_numpy(trainingnp_y))
        val_data = TensorDataset(torch.from_numpy(valnp_x), torch.from_numpy(valnp_y))

        train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size)
        val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size)

        newmodel.train()
        for i in range(epochs):
            for inputs, labels in train_loader:
                counter += 1
                inputs, labels = inputs.to(device), labels.to(device)
                newmodel.zero_grad()
                output = newmodel(inputs)
                loss = criterion(output.squeeze(), labels)
                loss.backward()
                nn.utils.clip_grad_norm_(newmodel.parameters(), clip)
                optimizer.step()
                
                # compare against the validation split at the step_size (print_every), to check for improvement
                if counter%print_every == 0:
                    val_losses = []
                    newmodel.eval()
                    for inp, lab in val_loader:
                        inp, lab = inp.to(device), lab.to(device)
                        out = newmodel(inp)
                        val_loss = criterion(out.squeeze(), lab)
                        val_losses.append(val_loss.item())

                    newmodel.train()
                    # slight mistake here, epochs will print as 1/1 always, because of i at the closest parent loop, but in fact it actually runs for 10 epochs as intended
                    print("Epoch: {}/{}...".format(i+1, epochs),
                          "Step: {}...".format(counter),
                          "Loss: {:.6f}...".format(loss.item()),
                          "Val Loss: {:.6f}".format(np.mean(val_losses)))
                    # if there is a improvement save the model
                    if np.mean(val_losses) <= valid_loss_min:
                        torch.save(newmodel.state_dict(), './state_dict_1.pt')
                        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                        valid_loss_min = np.mean(val_losses)

  6%|▌         | 3/50 [00:00<00:02, 22.93it/s]

Epoch: 1/1... Step: 50... Loss: 63331.775660... Val Loss: 3923.005707
Validation loss decreased (inf --> 3923.005707).  Saving model ...


  6%|▌         | 3/50 [00:00<00:01, 23.89it/s]

Epoch: 1/1... Step: 100... Loss: 18524.392262... Val Loss: 18289.451671


  6%|▌         | 3/50 [00:00<00:01, 25.32it/s]

...

Epoch: 1/1... Step: 59900... Loss: 6764.783198... Val Loss: nan


100%|██████████| 50/50 [00:02<00:00, 20.54it/s]
0it [00:00, ?it/s]
  6%|▌         | 3/50 [00:00<00:01, 26.41it/s]

Epoch: 1/1... Step: 59950... Loss: 38272.772239... Val Loss: nan


100%|██████████| 50/50 [00:02<00:00, 24.10it/s]
0it [00:00, ?it/s]


Epoch: 1/1... Step: 60000... Loss: 100261.797900... Val Loss: nan
